# Caiman: Motion correction

CaImAn is a Python toolbox for large scale Calcium Imaging data Analysis and behavioral analysis. The package has been developed at the Flatiron Institute by Pnevmatikakis, Giovannucci, Gunn and Friedrich ([Giovannucci et al, 2019](https://elifesciences.org/articles/38173)).

Information and examples of Caiman can be found in [Documentation](https://caiman.readthedocs.io/en/latest/) and [Caiman GitHub](https://github.com/flatironinstitute/CaImAn).

**Installation**

You can try to install Caiman in your environment with: `conda install -c conda-forge caiman`. If you cannot install caiman in your environment, create a new one `conda create -n caiman python=3.11 pip vs2019_win-64` or using mamba as suggested by the documentation: https://github.com/flatironinstitute/CaImAn

- **Instalation troubleshooting**
  - Update conda (recommended): `conda update conda`
  - If you use Windows 10, [disable the 260 character limit](https://github.com/tensorflow/tensorflow/issues/24835#issuecomment-453365761). 
  - Other packages that you may need to install are opencv: `pip install opencv-python`
  - Error "DLL load failed while importing defs", uninstall and install again the h5py package.
  - Error with 'tensorflow-base': try instead: `mamba create -n caiman --override-channels -c base -c conda-forge caiman`.

**Jupyter notebooks (Caiman GitHub)**:
- [Single-channel motion correction](https://github.com/flatironinstitute/CaImAn/blob/master/demos/notebooks/demo_motion_correction.ipynb)
- [Dual-channel motion correction](https://github.com/flatironinstitute/CaImAn/blob/master/demos/notebooks/demo_seeded_CNMF.ipynb)
- [Dendritic analysis](https://github.com/flatironinstitute/CaImAn/blob/master/demos/notebooks/demo_dendritic.ipynb)


# Example data

A time-series confocal recording of a pyramidal neuron dendrite with two sensors: GCaMP6s (channel 1) and mito-mCherry (channel 2):
* `dendrite_ch1.tif`
* `dendrite_ch2.tif`

# Import the packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tifffile

import time
import os

import caiman as cm
from caiman.motion_correction import MotionCorrect, tile_and_correct, motion_correction_piecewise
from caiman.source_extraction.cnmf import params as params
from caiman.paths import get_tempdir

# Paths

In [ ]:
# Change the paths and folder names according to your data structure
notebook_name = 'caiman_motion_correction'

# Data path to 'Data_example' folders. Change accordingly to your data structure.
data_path = os.path.dirname(os.getcwd())  # Moves one level up from the current directory

# Change the folder names accordingly
paths = {'data': data_path,
         'raw_data':  f'{data_path}/Data_examples/{notebook_name}/',
         'processed_data': f'{data_path}/Processed_data_examples/{notebook_name}/',
         'analysis': f'{data_path}/Analysis_examples/{notebook_name}/',         
         'plots': f'{data_path}/Analysis_examples/{notebook_name}/Plots/'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

In [ ]:
# Check the paths where you have to save the files
paths

## Caiman paths

The memory map files are usually saved in the current working (cwd) directory of the script. Although changing the paths of the [path functions](https://github.com/flatironinstitute/CaImAn/blob/main/caiman/paths.py) did not work, the following seems to work: 

- Change the environment variables (`os.environ`) and change the cwd directory. 


In [ ]:
os.chdir(paths['raw_data'])
os.getcwd()

In [ ]:
os.environ["CAIMAN_NEW_TEMPFILE"] = "true"
os.environ["CAIMAN_TEMP"] = paths['raw_data']
os.environ["CAIMAN_DATADIR"] = paths['raw_data']
get_tempdir()

# Load the file

For your data, use labs' data organization. Paths are defined in [My_project_main](Template_main.ipynb) and have to be loaded here.

In [ ]:
recording = 'dendrite'
file = f"{paths['raw_data']}/{recording}_ch1.tif"

stack_ch1 = cm.load(file)

## Play the movie (optional)

If you want to resize the file, create a new variable

In [ ]:
downsample_ratio = .2  # .2 means by a factor of 2
stack_ch1.resize(1, 1, downsample_ratio).play(
    q_max=99.5, fr=30, magnification=1)  # play movie (press q to exit)

# Parameters for motion correction

* [Motion correction parameters](https://github.com/flatironinstitute/CaImAn/blob/main/caiman/motion_correction.py) (function)
* [Tips for motion correction with Caiman](https://caiman.readthedocs.io/en/latest/CaImAn_Tips.html#motion-correction-tips)

Values in pixels

In [ ]:
# Run these lines only if you want to see the complete list of motion correction parameters
opts = params.CNMFParams()
opts.get_group('motion')

In [ ]:
max_shifts = (12, 12)    # maximum allowed rigid shift in pixels (view the movie to get a sense of motion). Default: 12, 12
strides =  (24, 24)      # create a new patch every x pixels for pw-rigid correction. Default: 24, 24
overlaps = (24, 24)      # overlap between patches (size of patch strides+overlaps). Default: 24, 24
num_frames_split = 300   # length in frames of each chunk of the movie (to be processed in parallel). Default: 100
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts. Default: 3
pw_rigid = False         # flag for performing rigid (False) or piecewise rigid motion correction (True). Default: false
shifts_opencv = True     # flag for correcting motion using bicubic interpolation (otherwise FFT interpolation is used). Default: Falase
border_nan = 'copy'      # replicate values along the boundary (if True, fill in with NaN). Default: copy

# Start a cluster
Parallel processing. Default: none.

**Clustering**. To enable parallel processing a (local) cluster needs to be set up. This is done with a cell below. The variable backend determines the type of cluster used. The default value 'multiprocessing' uses the multiprocessing package. The ipyparallel option is also available. More information on these choices can be found [here]. The resulting variable dview expresses the cluster option. If you use dview=dview in the downstream analysis then parallel processing will be used. [Documentation](https://github.com/flatironinstitute/CaImAn/blob/9b0b79ca61f20ce93259b9833e1fe18e26d4e086/docs/CLUSTER.md?plain=1#L22)

In [ ]:
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='multiprocessing', n_processes=None, single_thread=False)

# Create a motion correction object

Alternatively, you can create a dictionary with the parameters.

In [ ]:
mc_ch1 = MotionCorrect(stack_ch1, dview=dview, max_shifts=max_shifts,
                       strides=strides, overlaps=overlaps,
                       max_deviation_rigid=max_deviation_rigid, 
                       shifts_opencv=shifts_opencv, nonneg_movie=True,
                       border_nan=border_nan)

# Rigid motion correction
Simple rigid motion correction algorithm. For non-rigid correction, `pw_rigid = True`

In [ ]:
mc_ch1.motion_correct(save_movie=True)

# Load motion corrected movie

In [ ]:
# Load motion corrected movie
ch1_rig = cm.load(mc_ch1.mmap_file)
bord_px_rig = np.ceil(np.max(mc_ch1.shifts_rig)).astype(int)

## Plot the maximum projection

In [ ]:
plt.figure()
plt.imshow(mc_ch1.total_template_rig, cmap = 'gray')
plt.savefig(f"{paths['analysis']}{recording}_ch1_max_caiman.png")

## Inspect the corrected stack

Press "q" to stop and close the movie file

In [ ]:
ch1_rig.resize(1, 1).play(
    q_max=99.5, fr=300, magnification=1, bord_px = 0*bord_px_rig) # press q to exit

## Plot shifts computed by rigid registration

In [ ]:
plt.close()
plt.figure()
plt.plot(mc_ch1.shifts_rig)
plt.legend(['x shifts','y shifts'])
plt.xlabel('frames')
plt.ylabel('pixels')

plt.savefig(f"{paths['plots']}{recording}_caiman_shifts.png")

# Save the movie

**Bit depth**. save_memmap function converts uint16 input files into 'float32', which increases the file size 4 times. Be aware that if you convert the image to 16-bit again, some high intensity values can be generated from negative values. To avoid that, you can replace negative values with zeroes.

In [ ]:
ch1_rig.save(f"{paths['processed_data']}{recording}_ch1_caiman.tif")

## Save the movie as 16-bits

Convert the files back to 16-bit (first clip the negative values).

In [ ]:
# Convert and save to 16-bit tif
ch1_rig[ch1_rig < 0] = 0
ch1_rig_16bit = ch1_rig.astype('uint16')

# Save the 16-bit TIFF file
tifffile.imwrite(f"{paths['processed_data']}/{recording}_ch1_caiman16.tif",
                 ch1_rig_16bit)

# Apply correction to second channel

**Note**: If 'mmap_ch2' is not saved in the current working directory as expected, define the path to the actual file path (e.g. 'caiman/temp')

In [ ]:
# Load second channel file
ch2 = f"{paths['raw_data']}/{recording}_ch2.tif"

Note: If you have multiple files, then its recommended to save them in order 'F' and then reload them and save in order 'C' for efficient downstream processing.

In [ ]:
%%time
mmap_ch2 = mc_ch1.apply_shifts_movie(ch2, save_memmap=True, order='F')

In [ ]:
ch2_rig = cm.load(mmap_ch2)

## Save the corrected file from the second channel

Use the above code snippet to save as 16-bits if needed.

In [ ]:
ch2_rig.save(paths['processed_data']+f'{recording}_ch2_caiman.tif')

# Remove the memory map files

Manually or by using the below lines. Last memory map file has to be removed manually after closing or finishing the notebook. Note: I was not be able to close it within the code. 

In [ ]:
del ch2_rig

In [ ]:
os.remove(mc_ch1.mmap_file[0])

In [ ]:
os.remove(mmap_ch2)

# Quality control analysis

In [ ]:
stack_orig = cm.load(f"{paths['raw_data']}/{recording}_ch1.tif")
stack_caiman = cm.load(f"{paths['processed_data']}/{recording}_ch1_caiman16.tif")

In [ ]:
fig, ax = plt.subplots()

ax.hist(stack_orig.flatten(), density= True, bins = 300, cumulative=True, 
        histtype='step', fill=False, edgecolor='gray', label='original')

ax.hist(stack_caiman.flatten(), density= True, bins = 300, cumulative=True, 
        histtype='step', fill=False, edgecolor='magenta', label='caiman')

ax.set_ylabel('Normalized count')
ax.set_xlabel('Pixel value')
ax.legend()

fig.savefig(f"{paths['plots']}{recording}_ch1_histo.png")